In [ ]:
import multiprocessing
from GCForest import gcForest
import pandas as pd
import numpy as np
import pickle 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import Imputer
import warnings
warnings.filterwarnings("ignore")
#用来计算程序运行时间
import datetime
starttime = datetime.datetime.now()

In [ ]:
#读取Y
all_df_y_2c = pickle.load(open("./dump_file/all_df_y_2c","rb"))
print("Y.shape:",all_df_y_2c.shape)
all_df_y = pickle.load(open("./dump_file/all_df_y","rb"))
all_df_y['2cArousal'] = 0
all_df_y['2cArousal'][all_df_y['valence'] >= 5] = 1
all_df_y['2cValence'] = 0
all_df_y['2cValence'][all_df_y['valence'] >= 5] = 1
print(all_df_y.head(5))

#读取32个通道的EEG数据，每个通道包含32×40=1280个信号样本（人次×每人次40实验）
#每个样本向量大小为8064点（63s*128Hz）
for eegCH in range(1,33,1):
    #file_path = "./dump_file/CH{}_df_EEG_x".format(eegCH)
    #locals()['CH{}_df_EEG_x'.format(eegCH)] = pickle.load(open(file_path,"rb"))
    file_path = "./dump_file/CH{}eegfft_df".format(eegCH)
    locals()["CH{}eegfft_df".format(eegCH)] = pickle.load(open(file_path,"rb"))
'''
X = CH1eegfft_df
y = all_df_y[["2cArousal"]]
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3,stratify=y,random_state=2018)   
xTrainIdx = X_tr.index
xTestIdx = X_te.index
'''
y = all_df_y[["2cValence"]]
xTrainIdx = pickle.load(open("./dump_file/xTrainIdx","rb"))
xTestIdx = pickle.load(open("./dump_file/xTestIdx","rb"))
y_tr = y.loc[xTrainIdx]
y_te = y.loc[xTestIdx]

In [ ]:
for eegCH in range(1,33,1):
    for i in range(0,1280,1):
for eegCH in range(1,33,1): 
    #时域数据
    #locals()['CH{}TrainSet'.format(eegCH)] = locals()['CH{}_df_EEG_x'.format(eegCH)].loc[xTrainIdx]
    #locals()['CH{}TestSet'.format(eegCH)] = locals()['CH{}_df_EEG_x'.format(eegCH)].loc[xTestIdx]
    #频域数据
    locals()['CH{}TrainSet'.format(eegCH)] = locals()['CH{}eegfft_df'.format(eegCH)].loc[xTrainIdx]
    locals()['CH{}TestSet'.format(eegCH)] = locals()['CH{}eegfft_df'.format(eegCH)].loc[xTestIdx]

In [ ]:
myWindowsSize = 256
myStrideSize = 64
gcf = gcForest(shape_1X=8064, window=myWindowsSize, stride=myStrideSize,tolerance=0.0,n_cascadeRF=1, 
               min_samples_mgs=0.1, min_samples_cascade=0.1,n_jobs=19)
if True:
    for eegCH in range(1,33,1): 
        print("CH{} running multi-grain scan".format(eegCH))
        xTrain,yTrain = locals()['CH{}TrainSet'.format(eegCH)].values,y_tr.values
        xTest = locals()['CH{}TestSet'.format(eegCH)].values   
        locals()['CH{}mgsTrainVector'.format(eegCH)] = gcf.mg_scanning(xTrain,yTrain)
        locals()['CH{}mgsTestVector'.format(eegCH)] = gcf.mg_scanning(xTest)
        filePath = "./dump_file_V2/CH{}mgsTrainVector_{}_{}".format(eegCH,myWindowsSize,myStrideSize)
        pickle.dump(locals()['CH{}mgsTrainVector'.format(eegCH)],open(filePath,"wb"))
        filePath = "./dump_file_V2/CH{}mgsTestVector_{}_{}".format(eegCH,myWindowsSize,myStrideSize)
        pickle.dump(locals()['CH{}mgsTestVector'.format(eegCH)],open(filePath,"wb"))
else:
    for eegCH in range(1,33,1):
        filePath = "./dump_file_V2/CH{}mgsTrainVector_{}_{}".format(eegCH,myWindowsSize,myStrideSize)
        locals()['CH{}mgsTrainVector'.format(eegCH)] = pickle.load(open(filePath,"rb"))
        filePath = "./dump_file_V2/CH{}mgsTestVector_{}_{}".format(eegCH,myWindowsSize,myStrideSize)
        locals()['CH{}mgsTestVector'.format(eegCH)]= pickle.load(open(filePath,"rb"))